In [5]:
import numpy as np
import scipy.stats as stats
import math
import pandas as pd
import statsmodels.api as sm
from itertools import combinations
import seaborn as sns
import folium
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize

In [4]:
"""
This is called "filtered_projects" because I took the original china-seaport-dataset and removed
anything that's not Africa or South America. I could have done this purely in the dataframe but
to me this was easier.
"""
filtered_projects = pd.read_csv("../data/china-seaport-dataset-filtered-china-south-america.csv")

filtered_projects.sample(5)

,Unnamed: 0,port_name,recipient,region,commitment_year,aiddata_tuff_project_id,recommended_for_aggregates,umbrella,financier_country,implementation_start_year,...,funding_agencies,cofinancing_agencies,receiving_agencies,implementing_agencies,accountable_agencies,amount_constant_usd2021,latitude,longitude,osm_link,precision_level
51,113,Sierra Leone Fish Harbour,Sierra Leone,Africa,2019,62533,Yes,No,China (People's Republic of),0,...,China Ministry of Commerce,NaN,Government of Sierra Leone|Sierra Leone Minist...,Zhejiang Communications Construction Group Co Ltd,NaN,7.362901e+07,8.272123,-13.167560,NaN,approximate location
35,83,Port of Goubet,Djibouti,Africa,2012,30346,Yes,No,China (People's Republic of),2013,...,Export-Import Bank of China (China Eximbank),NaN,Djiboutian Ministry of Economy and Finance,"China Harbour Engineering Co., Ltd. (CHEC)",NaN,7.928452e+07,11.535403,42.520562,https://www.openstreetmap.org/relation/11199579,precise
29,75,Port of Bata,Equatorial Guinea,Africa,2007,1056,Yes,No,China (People's Republic of),2008,...,Export-Import Bank of China (China Eximbank),NaN,Government of Equatorial Guinea,CCCC First Harbor Engineering Company Ltd.|Chi...,NaN,6.593594e+08,1.819832,9.749290,https://www.openstreetmap.org/relation/9102676,precise
40,97,Port of Takoradi,Ghana,Africa,2011,73196,No,No,China (People's Republic of),0,...,China Development Bank (CDB),NaN,Government of Ghana,"China Harbour Engineering Co., Ltd.",NaN,0.000000e+00,4.887073,-1.735196,https://www.openstreetmap.org/way/756865759,boundary not clear
44,103,Porto Sao Luis,Brazil,South America,2018,62973,No,No,China (People's Republic of),2018,...,Industrial and Commercial Bank of China (ICBC),NaN,"China Communications Construction Co., Ltd. (C...","China Communications Construction Co., Ltd. (C...",NaN,0.000000e+00,-2.573857,-44.365872,https://www.openstreetmap.org/relation/10189309,precise
